In [1]:
import geopandas as gpd
from shapely.geometry import Point
from shapely.ops import nearest_points
from pyproj import Transformer
from shapely import Point, wkt
import pandas as pd

In [2]:
# Load the shapefiles
dams_csv = pd.read_csv(r"D:\CS\GitHub\Nuclear-Site-Selection\dam\dam.csv")
dams = gpd.read_file(r"D:\CS\GitHub\Nuclear-Site-Selection\dam\Dam.shp")
airports = gpd.read_file(r'D:\CS\GitHub\Nuclear-Site-Selection\Airport\Airport.shp')

In [3]:
transformer = Transformer.from_crs("epsg:4326", "epsg:3857", always_xy=True)

In [4]:
dams = dams.to_crs(epsg=4326)
airports = airports.to_crs(epsg=4326)

def find_nearest_airport_and_distance(dam_point, airports_gdf):
    nearest_geom = nearest_points(dam_point, airports_gdf.unary_union)[1]
    nearest_airport = airports_gdf.loc[airports_gdf.geometry == nearest_geom]
    
    # Convert the coordinates to EPSG:3857 for distance calculation
    dam_point_proj = transformer.transform(dam_point.x, dam_point.y)
    nearest_geom_proj = transformer.transform(nearest_geom.x, nearest_geom.y)
    
    # Calculate the distance in meters
    distance_m = ((dam_point_proj[0] - nearest_geom_proj[0]) ** 2 + (dam_point_proj[1] - nearest_geom_proj[1]) ** 2) ** 0.5
    
    # Convert the distance to kilometers
    distance_km = distance_m / 1000
    
    return nearest_airport.iloc[0], distance_km


In [5]:
nearest_airports_and_distances = dams.geometry.apply(lambda dam: find_nearest_airport_and_distance(dam, airports))


In [125]:
dams['nearest_airport'] = nearest_airports_and_distances.apply(lambda x: x[0]['name'])  # Replace 'name' with the correct column name
dams['distance_to_nearest_airport_km'] = nearest_airports_and_distances.apply(lambda x: x[1])

In [126]:
dams

,dm_name,dm_lat,dm_long,dtcode,STATE_1,nearest_ai,distance_t,geometry,nearest_airport,distance_to_nearest_airport_km
0,Ajan Dam,25.033185,86.295520,Jamui,BIHAR,"Atal Bihari Vajpayee International Airport, Si...",85.013332,POINT (86.29552 25.03319),"Atal Bihari Vajpayee International Airport, Si...",85.013332
1,Amrity Dam,24.889893,86.101464,Jamui,BIHAR,"Atal Bihari Vajpayee International Airport, Si...",86.238498,POINT (86.10146 24.88989),"Atal Bihari Vajpayee International Airport, Si...",86.238498
2,Badua Dam,24.887072,86.645950,Banka,BIHAR,"Atal Bihari Vajpayee International Airport, Si...",54.301341,POINT (86.64595 24.88707),"Atal Bihari Vajpayee International Airport, Si...",54.301341
3,Ahraura Dam,25.000400,83.035647,Mirzapur,UTTAR PRADESH,"Lal Bahadur Shastri International Airport, Var...",58.654502,POINT (83.03565 25.00040),"Lal Bahadur Shastri International Airport, Var...",58.654502
4,Maudaha (Swami Brahmanand ) Dam,25.588776,79.707894,Hamirpur,UTTAR PRADESH,"Khajuraho Airport, Khajuraho",98.438675,POINT (79.70789 25.58878),"Khajuraho Airport, Khajuraho",98.438675
...,...,...,...,...,...,...,...,...,...,...
5614,Ghoghri Dam,21.722910,77.931928,Betul,MADHYA PRADESH,"Dr. Babasaheb Ambedkar International Airport, ...",146.050562,POINT (77.93193 21.72291),"Dr. Babasaheb Ambedkar International Airport, ...",146.050562
5615,Kesli Dam,23.400320,78.790828,Sagar,MADHYA PRADESH,"Jabalpur Airport, Jabalpur",143.543884,POINT (78.79083 23.40032),"Jabalpur Airport, Jabalpur",143.543884
5616,Sonpur Dam,23.388370,78.707888,Sagar,MADHYA PRADESH,"Lalitpur Airport, Lalitpur",164.609632,POINT (78.70789 23.38837),"Lalitpur Airport, Lalitpur",164.609632
5617,Surajpura Dam,23.515067,78.740555,"Kesli, Sagar",MADHYA PRADESH,"Lalitpur Airport, Lalitpur",150.367778,POINT (78.74056 23.51507),"Lalitpur Airport, Lalitpur",150.367778


In [ ]:
dams.to_file(r"D:\CS\GitHub\Nuclear-Site-Selection\dam\Dam.shp")

C:\Users\Nirman\AppData\Local\Temp\ipykernel_9184\1224098203.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  dams.to_file(r"D:\CS\GitHub\Nuclear-Site-Selection\dam\Dam.shp")


In [134]:
dams_csv['nearest_airport'] = nearest_airports_and_distances.apply(lambda x: x[0]['name'])  # Replace 'name' with the correct column name
dams_csv['distance_to_nearest_airport_km'] = nearest_airports_and_distances.apply(lambda x: x[1])

In [136]:
dams_csv

,dm_name,STATE_1,dm_lat,dm_long,dm_ses_zon,geometry,Avg_population,nearest_airport,distance_to_nearest_airport_km
0,Ajan Dam,BIHAR,25.033185,86.295520,Seismic Zone-IV,POINT (4622889.0973000005 4126138.3808999993),538.215336,"Atal Bihari Vajpayee International Airport, Si...",85.013332
1,Amrity Dam,BIHAR,24.889893,86.101464,Seismic Zone-IV,POINT (4604374.947700001 4109742.2074999996),831.957676,"Atal Bihari Vajpayee International Airport, Si...",86.238498
2,Badua Dam,BIHAR,24.887072,86.645950,Seismic Zone-IV,POINT (4658284.6098000035 4111875.8922000006),735.990780,"Atal Bihari Vajpayee International Airport, Si...",54.301341
3,Ahraura Dam,UTTAR PRADESH,25.000400,83.035647,Seismic Zone-III,POINT (4300505.322700001 4111895.7787999995),525.465066,"Lal Bahadur Shastri International Airport, Var...",58.654502
4,Maudaha (Swami Brahmanand ) Dam,UTTAR PRADESH,25.588776,79.707894,Seismic Zone-II,POINT (3971214.0732000023 4172614.835000001),256.918575,"Khajuraho Airport, Khajuraho",98.438675
...,...,...,...,...,...,...,...,...,...
5614,Ghoghri Dam,MADHYA PRADESH,21.722910,77.931928,NaN,POINT (3790043.5779 3754215.767499998),117.735819,"Dr. Babasaheb Ambedkar International Airport, ...",146.050562
5615,Kesli Dam,MADHYA PRADESH,23.400320,78.790828,NaN,POINT (3878802.2476999983 3935394.917800002),206.101857,"Jabalpur Airport, Jabalpur",143.543884
5616,Sonpur Dam,MADHYA PRADESH,23.388370,78.707888,NaN,POINT (3870477.265900001 3934170.967799999),172.484460,"Lalitpur Airport, Lalitpur",164.609632
5617,Surajpura Dam,MADHYA PRADESH,23.515067,78.740555,NaN,POINT (3873874.817599997 3947900.0031999983),335.399006,"Lalitpur Airport, Lalitpur",150.367778


In [137]:
dams_csv.to_csv(r"D:\CS\GitHub\Nuclear-Site-Selection\dam\dam.csv")